In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from queue import Queue
import random

In [2]:
def create_dag(max_level=10, max_children=10, max_number_nodes=10, having_childs_prob=2):
    q = Queue()
    last_node = 0
    G = nx.Graph()
    G.add_node(0)
    q.put((0, 0))

    weights = [1/n for n in range(1, max_children+1)]

    while not q.empty():
        node, level = q.get()

        if not level == max_level and last_node <= max_number_nodes:
            has_childs = random.randint(1, having_childs_prob)

            if node == 0 or has_childs != 1:
                
                n_childs = random.choices(range(1, max_children + 1))[0]

                for c in range(n_childs):
                    # Nodes start from 0
                    if last_node == max_number_nodes - 1:
                        break

                    last_node += 1
                    G.add_node(last_node)
                    G.add_edge(node, last_node)
                    q.put((last_node, level + 1))

    return G

In [55]:
def create_dag_levelist(max_level=10, max_level_nodes=10 , max_number_nodes=10, having_childs_prob=2):
    q = Queue()
    last_node = 0
    last_level = 0
    nodes = [[0]]

    G = nx.DiGraph()
    G.add_node(0)

    q.put((last_node, last_level))

    while not q.empty():
        node, level = q.get()

        # Adding nodes to the new level
        if last_level == level and last_level < max_level:
            last_level += 1
            n_new_level_nodes = random.randint(1, max_level_nodes)
            new_level_nodes = []

            for i in range(0, n_new_level_nodes - 1 ):
                if last_node >= max_number_nodes:
                    break

                last_node += 1
                G.add_node(last_node)
                q.put((last_node, last_level))
                new_level_nodes.append(last_node)
            
            nodes.append(new_level_nodes)

        # Add edges if the current node has node incoming edges
        if node != 0 and G.in_degree(node) == 0:
            previous_level_nodes = nodes[level-1]
            n_parents = random.randint(1, previous_level_nodes.__len__())
            parent = random.sample(previous_level_nodes, n_parents)

            for p in parent:
                G.add_edge(p, node)


        # Adding edges for the current node
        if level < last_level:
            next_level_nodes = nodes[level+1]
            has_child = random.randint(1, having_childs_prob)
            if node == 0 and (next_level_nodes.__len__() > 0) or (has_child != 1 and next_level_nodes.__len__() > 0):
                # weights = [1/n for n in range(1, next_level_nodes.__le__())]
                # n_childs = random.choices(1, next_level_nodes.__len__(), weights=weights)[0]
                n_childs = random.randint(1, next_level_nodes.__len__())
                children = random.sample(next_level_nodes, n_childs)

                for child in children:
                    G.add_edge(node, child)

    return G


In [56]:
def adj_matrix(g:nx.Graph):
    n_nodes = g.number_of_nodes()
    adj = np.full((n_nodes, n_nodes), -1)
    nodes = g.nodes
    
    for node in nodes:
        edges = g.edges(node)
        for u, v  in edges:
            adj[u, v] = 0

    return adj


In [58]:
# g = create_dag(max_level=5, max_children=5, max_number_nodes=16, having_childs_prob=4)
g = create_dag_levelist(max_level=5, max_level_nodes=5, max_number_nodes=25, having_childs_prob=3)
g.number_of_nodes(), g.number_of_edges()

(13, 16)

In [59]:
g.edges

OutEdgeView([(0, 3), (0, 1), (0, 2), (3, 4), (4, 6), (4, 7), (4, 5), (5, 10), (6, 9), (6, 8), (6, 11), (6, 10), (7, 10), (8, 12), (10, 12), (11, 12)])

In [60]:
adj_matrix(g)

array([[-1,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])